# Landsat for the classification of the urban and agriculture areas 

**Set the environment**

Initialize Earth Engine account

In [1]:
import os
import ee
from ee import batch
import json
import requests
import folium
import urllib
import numpy as np
import pandas as pd
from IPython.display import display, Image
import matplotlib.pyplot as plt
from datetime import timedelta, date

ee.Initialize()

**Functions**

In [2]:
def display_image(image, region, Vizz = None):
    """
    Displays images in notebook
    """ 
    ## Visualization
    if Vizz:
        image = image.visualize(**Vizz)
        
    visual = Image(url=image.getThumbUrl({
                'region':region
                }))
    
    display(visual)

In [3]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    #If the cloud bit (5) is set and the cloud confidence (7) is high
    #or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3));
    #Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud.Not()).updateMask(mask2);

In [4]:
def Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale, bandNames = None):
    ## Define your collection
    collection = ee.ImageCollection(Collection_id)

    ## Filter 
    collection = collection.filterBounds(geom).filterDate(startDate,stopDate)\
            .map(cloudMaskL457)

    ## Composite
    composite = collection.median()

    ## Choose the scale
    composite =  composite.reproject(crs='EPSG:4326', scale=scale)

    ## Select the bands
    if bandNames:
        composite = composite.select(bandNames)
    
    return composite

## Landsat 7 

### USGS Landsat 7 Surface Reflectance Tier 1 ([gee](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_SR))
**Dataset Availability**: January 1999 - Present

**Wavebands**

|Name	      |Resolution	|Wavelength	      |Description|
|-------------|-------------|-----------------|-----------|
|B1	          |30 meters	|0.45 - 0.52 µm	  |Blue                                                  |
|B2	          |30 meters	|0.52 - 0.60 µm	  |Green                                                 |
|B3	          |30 meters	|0.63 - 0.69 µm	  |Red                                                   |
|B4	          |30 meters	|0.77 - 0.90 µm	  |Near infrared                                         |
|B5	          |30 meters	|1.55 - 1.75 µm	  |Shortwave infrared 1                                  |
|B6	Kelvin	  |             |0.40 - 12.50 μm  |Band 6 brightness temperature                         |
|B7		 	  |             |2.08 - 2.35 μm	  |Band 7 (shortwave infrared 2) surface reflectance     |
|sr_atmos_opacity|          |                 |Atmospheric opacity                                   |
|sr_cloud_qa|               |                 |Cloud quality attributes                              |

In [14]:
# GEE Image Collection ID
Collection_id = 'LANDSAT/LE07/C01/T1_SR'
# Area of Interest (AoI)
geom = ee.Geometry.Point(-120.7224, 37.3872).buffer(10000)
region = geom.bounds().getInfo()['coordinates']
# Start and stop of time series
startDate = ee.Date('2014')
stopDate  = ee.Date('2017')
# Scale in meters
scale = 30

**Cloud Free Composite**

RGB

In [15]:
# Area of Interest (AoI)
vis = {'min':0,'max':3000, 'gamma':1.4, 'bands':['B3', 'B2', 'B1']}
image = Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale)
display_image(image, region, Vizz = vis)

In [16]:
# Area of Interest (AoI)
geom = ee.Geometry.Point(-112.6799, 42.9816).buffer(10000)
region = geom.bounds().getInfo()['coordinates']
vis = {'min':0,'max':3000, 'gamma':1.4, 'bands':['B3', 'B2', 'B1']}
image = Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale)
display_image(image, region, Vizz = vis)

NDVI

In [17]:
palette = ['blue', 'white', 'green']
vis = {'min': -0.5, 'max': 0.5, 'bands':'nd', 'palette': palette}
#Calculate NDVI
image_ndvi = image.normalizedDifference(['B4','B3'])
display_image(image_ndvi, region, Vizz = vis)

## Cropland Data Layers
### USDA NASS Cropland Data Layers ([gee](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL))

**Dataset Availability**: January 1997 - Present

**Resolution**
30 meters

**Bands**

|Name 	    |Min|Max |Description 	|
|-----------|---|----|--------------|
|cropland 	|1 	|254 |Main crop-specific land cover classification.|
|cultivated |1 	|2   |Classification layer for identifying cultivated and non-cultivated land cover. Available from 2013 to 2017.|
|confidence |0 	|100 |Per-pixel predicted confidence of the given classification, with 0 being the least confident and 100 the most confident. Available from 2008 to 2017 (Note: Confidence for Florida and Washington D.C. is unavailable for 2010).|

In [9]:
# GEE Image Collection ID
Collection_id = 'USDA/NASS/CDL'
# Start and stop of time series
startDate = ee.Date('2014')
stopDate  = ee.Date('2017')

In [10]:
dataset = ee.ImageCollection(Collection_id)\
    .filterBounds(geom)\
    .filterDate(startDate,stopDate)

# First image
image = ee.Image(dataset.first())

# Choose the scale
image =  image.reproject(crs='EPSG:4326', scale=scale)

In [11]:
vis = {'min':1,'max':254, 'bands':'cropland'}
display_image(image, region, Vizz = vis)